In [ ]:
import torch
import numpy as np
from omegaconf import OmegaConf

from physicsnemo.sym.models.fully_connected import FullyConnectedArch
from physicsnemo.sym.key import Key
from physicsnemo.sym.utils.io.vtk import var_to_polyvtk,grid_to_vtk

from three_fin_geometry import ThreeFin

# 1. 載入 config
cfg = OmegaConf.load("conf/conf_thermal.yaml")
cfg.run_mode = "eval"

# 2. 建立 input keys
input_keys = [
    Key("x"), Key("y"), Key("z"),
    Key("fin_height_s"),
    Key("fin_length_s"),
    Key("fin_thickness_s"),
]

# 3. 建立三個網路架構
flow_net = FullyConnectedArch(
    input_keys=input_keys, output_keys=[Key("u"), Key("v"), Key("w"), Key("p")]
)
thermal_f_net = FullyConnectedArch(input_keys=input_keys, output_keys=[Key("theta_f")])
thermal_s_net = FullyConnectedArch(input_keys=input_keys, output_keys=[Key("theta_s")])

# 4. 載入模型權重（指定路徑）
base_path = "/workspace/physicsnemo-sym/examples/three_fin_3d_0731_inf/outputs/three_fin_thermal/"

flow_net.load_state_dict(torch.load(base_path + "flow_network.0.pth", map_location="cpu"))
thermal_f_net.load_state_dict(torch.load(base_path + "thermal_f_network.0.pth", map_location="cpu"))
thermal_s_net.load_state_dict(torch.load(base_path + "thermal_s_network.0.pth", map_location="cpu"))

flow_net.eval()
thermal_f_net.eval()
thermal_s_net.eval()

# 5. 建立幾何 sample 點（使用你的參數）
geo = ThreeFin(parameterized=True)
param = {
    "fin_height_s": 0.030,
    "fin_length_s": 0.195,
    "fin_thickness_s": 0.0025,
}

In [ ]:
# 1️⃣ 流體區域推論
invar_fluid = geo.geo.sample_interior(nr_points=100000, parameterization=param, quasirandom=True)
invar_fluid_tensor = {k: torch.tensor(v, dtype=torch.float32) for k, v in invar_fluid.items()}
with torch.no_grad():
    pred_flow = flow_net(invar_fluid_tensor)
    pred_theta_f = thermal_f_net(invar_fluid_tensor)

pred_fluid_np = {
    **invar_fluid,
    **{k: v.cpu().numpy() for k, v in pred_flow.items()},
    **{k: v.cpu().numpy() for k, v in pred_theta_f.items()},
}

# 加入 velocity 欄位，用於 streamline
velocity = np.stack([
    pred_fluid_np["u"],
    pred_fluid_np["v"],
    pred_fluid_np["w"]
], axis=-1)
pred_fluid_np["velocity"] = velocity

# ➤ 輸出 fluid_only.vtp
var_to_polyvtk(pred_fluid_np, "fluid_only.vtp")

In [ ]:
from physicsnemo.sym.utils.io.vtk import VTKFromFile
from physicsnemo.sym.domain.inferencer  import PointVTKInferencer
import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np
from sympy import Symbol, Eq, tanh, Or, And
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra.utils import to_absolute_path

import physicsnemo.sym
from physicsnemo.sym.hydra import to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.utils.io import csv_to_dict
from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.geometry.primitives_3d import Box, Channel, Plane
from physicsnemo.sym.models.fourier_net import FourierNetArch
from physicsnemo.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)
from physicsnemo.sym.domain.monitor import PointwiseMonitor
from physicsnemo.sym.domain.inferencer import PointVTKInferencer
from physicsnemo.sym.utils.io import (
    VTKUniformGrid,
)
from physicsnemo.sym.key import Key
from physicsnemo.sym.node import Node
from physicsnemo.sym.eq.pdes.navier_stokes import NavierStokes
from physicsnemo.sym.eq.pdes.turbulence_zero_eq import ZeroEquation
from physicsnemo.sym.eq.pdes.basic import NormalDotVec, GradNormal
from three_fin_geometry import ThreeFin